Nom Etudiant 1: <font color='blue'>PARNICHTHANAKOM Pornpround</font>

# TME 2.1 : Projet Detection de motifs - Hash Table

Nous allons développer des algorithmes pour chercher de motifs dans des séquences d'ADN. Pour commencer, nous allons générer des données artificielles qui nous permettront de tester rapidement nos algorithmes (**Partie A**). Ensuite, nous allons chercher les motifs dans le génome _C. glabrata_ et analyser les résultats (**Partie B**). Finalement, nous allons appliquer les algorithmes developées pour chercher de motifs dans les données de ChipSeq de _C. glabrata_ (**Partie C**).

## Partie A : Recherche de pattern (motifs) identiques

1\. Faites une fonction pour générer aléatoirement des séquences artificielles. Utilisez les fonctions de la bibliothèque `random`. Toutes les lettres peuvent être équiprobables pour la génération des séquences. Si le nombre de séquences est égal à 1 (`n = 1`), renvoyez une chaîne (string) de caractères au lieu d'une liste.

In [21]:
import random

nuc = ('A', 'C', 'G', 'T')

# Pour faire un test simple
n = 3 # nombre de séquences
t = 8 # taille des séquences

def generateRandomSequences(n:int, t:int, upper = True):
    """
    Génére plusieurs séquences nucléotidiques aléatoires 
    entrée n : nombre de séquences
    entrée t : taille des séquences
    entrée upper : bool, si True, les nucléotides seront majuscule, False, minuscule
    sortie séquences : liste de séquences nucléotidiques aléatoires ou un string si n = 1
    """
    
    sequences = []

    for i in range(n) :
        aux = ""
        for i in range(t) :
            aux += random.choice(nuc)
        if upper : sequences.append(aux.upper())
        else : sequences.append(aux.lower())
    
    if n == 1 :
        return sequences[0]
    
    return sequences

generateRandomSequences(n = n, t = t, upper = True)

['CCTAGAAG', 'GATGGTCT', 'GTTCGGAT']

In [22]:
type(generateRandomSequences(n = 1, t = t, upper = True))

str

2\. Puis, générez `n = 10` séquences de `t = 41` nucléotides chacune. Pour obtenir le même résultat dans le développement de ce TME, choisissez un "seed", mais différent de celui de vos camarades. Pensez à changer le `seed` pour effectuer des tests variés.

In [24]:
# Pour obtenir les séquences artificielles
n = 10
t = 41

seed = 3582 

random.seed(seed)

seqs = generateRandomSequences(n,t)

for seq in seqs:
    print(seq)

ACATCCCTAGAAGGATGGTCTGTTCGGATCTGTTTCTTAGA
GTGCCACCTACACACTAACCCATTGGTGGTAACCACTCTGA
CTATTCGTGTAATAAGACAAGACGGCTGGATCGGCTTCTCT
CGAGGGATTACCTGAGAGGTAGTGTTGGCCGTCGATTTAAT
GAGCTTGTGTTTATGCAGTACGTCAGGAAATCTCAGCCATT
TCTTGCTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCG
GTGGCGGGATCAGGGGCAGTTCCTCATCTACTGCGGCAGTG
GGGTCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA
CGTCAGAGAGCTACCAGTAATTAACATGCCAATCGGAGTGT
TCTTCGTCACGAGCCTTCTAGAGAAATTGATGCAGCTCGTC


3\. Nous allons maintenant implanter dans les séquences artificielles generés avant un motif de taille `k = 4` à des positions aléatoires (choisies uniformément le long de la séquence). Controlez le `upper` pour visualiser le motif. **BONUS :** on considère une proportion `f = 0.9` de séquences qui possèdent le motif.

In [25]:
k = 4 # taille des motifs
# Pour le bonus
f = 0.9 # fréquence d'occurences des motifs

def implantMotifs(motifs:str, sequences:list, f = 1):
    """
    Insére un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée séquences : liste de séquences
    entrée f : fréquence d'implantation, si 1, toutes les séquences contiendront un motif
    sortie modified_sequences: liste de séquences ayant le motif implanté
    """

    if f == 1 :
        modified_sequences = []
        for seq in sequences : 
            pos = random.randint(0,len(seq)-1)
            modified_sequences.append(seq[:pos].upper()+motifs.lower()+seq[pos:].upper())
        return modified_sequences
    
    not_implanted = sequences
    implanted = []
    while len(implanted)/len(sequences) < f :
        seq = random.choice(not_implanted)
        not_implanted.remove(seq)
        pos = random.randint(0,len(seq)-1)
        implanted.append(seq[:pos].upper()+motifs.lower()+seq[pos:].upper())
      
    return implanted + not_implanted

# Pour obtenir le même résultat que dans le dévelopent de ce TME
random.seed(seed)

motif = generateRandomSequences(1,4)
seqs_implanted = implantMotifs(motif,seqs,1)

print("Le motif géneré est : ", motif)

for seq in seqs_implanted:
    print(seq)

Le motif géneré est :  ACAT
ACATCCCTAGAAGGATGGTCTGTTCGGATCTGTTTCacatTTAGA
GTGCCACCTacatACACACTAACCCATTGGTGGTAACCACTCTGA
CTATTCGTGTAATAacatAGACAAGACGGCTGGATCGGCTTCTCT
CGAGGGATTACCacatTGAGAGGTAGTGTTGGCCGTCGATTTAAT
GAGCTTGTGTTTATGCAGTACGTCAGacatGAAATCTCAGCCATT
TCTTGCacatTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCG
GTGGCGGGATCAGGGGCacatAGTTCCTCATCTACTGCGGCAGTG
GGGTacatCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA
CGTCAGAGAGCTACCAGTAATTAACATGCCAATCacatGGAGTGT
acatTCTTCGTCACGAGCCTTCTAGAGAAATTGATGCAGCTCGTC


4\. Faites une fonction pour chercher les $m$ motifs de taille $k$ les plus fréquents dans l'ensemble des séquences.

In [26]:
def searchMotifs(k:int, sequences:list):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de séquences
    sortie motifs: dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    
    motifs  = {}

    for seq in sequences :
        for i in range(len(seq)-k+1) :
            aux = seq[i:i+k]
            if aux in motifs : motifs[aux] += 1
            else : motifs[aux] = 1
    return motifs

test_motifs = searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])

print(test_motifs)

{'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}


5\. Faite aussi une fonction qu'affiche les `top` motifs les plus fréquents. Cette fonction peut également trier le dictionnaire de manière croissant ou décroissant.

In [30]:
def getTopMotifs(motifs:dict, top, decroissant = True):
    """
    Renvoie les top motifs le plus fréquent
    entrée motifs: dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    entrée top : les top plus fréquent, si top == "all", ne filtrez pas et renvoyer le dictionnaire complet trié
    entrée decroissant : bool, si True le dictionnaire est trié par ordre décroissant de valeur
    sortie motifsfreq: dictionnaire contenant les top motifs les plus fréquents, clé = motif, valeur = fréquence d'observation
    >>>getTopMotifs({'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}, 2)
    {'TAA': 3, 'TAT': 2}
    """

    lst = [(key,freq) for key,freq in motifs.items()]
    lst.sort(key=lambda x : x[1], reverse=decroissant)

    if top == "all" : return dict(lst)
    return dict(lst[:top])
    
test_motifs_top = getTopMotifs(test_motifs, 2, decroissant = True)
    
print(test_motifs_top)

{'TAA': 3, 'TAT': 2}


6\. Testez cette fonction sur un l'ensemble de séquences articielles avec le motif implanté généré dans les questions précédentes. Faites attention à ne pas comparer des séquences en majuscules avec des minuscules. Avez-vous trouvé le motif implanté ? Si oui, à quelle fréquence ?

In [31]:
seqs_implanted_motifs = [seq.upper() for seq in seqs_implanted]
seqs_implanted_motifs_top = getTopMotifs(searchMotifs(4,seqs_implanted_motifs),5,True)

print("Le motif géneré est : ", motif)

print(seqs_implanted_motifs_top)

print("Fréquence observé : ", seqs_implanted_motifs_top[motif])

Le motif géneré est :  ACAT
{'ACAT': 12, 'CACA': 6, 'CATT': 6, 'CGTC': 6, 'GGAT': 5}
Fréquence observé :  12


7\. Quell est la complexité de votre algorithme?

<font color='blue'>
Votre réponse
</font>

8\. Certains motifs sont reverse complémentaires. Implantez ces motifs reverse complémentaires dans les séquences artificielles déjà générées, puis recherchez ces motifs dans les séquences. **Attention** : vous devez réutiliser les fonctions développées précédemment. Il n’est pas nécessaire de définir de nouvelles fonctions.

Pour cela, nous allons développer un module **Python** appelé **motifsSearch**.

* Créez un fichier nommé `motifsSearch.py`.
* Copiez et collez les fonctions déjà développées dans ce fichier texte.
* Sauvegardez-le sous ce nom dans le même répertoire que les fichiers **Jupyter** où vous allez l'utiliser.

In [ ]:
# Reutiliser la fonction déjà defini (TME1.2_ORFs, Question 3)
from motifsSearch import reversecompl

motif_rv # Code

seqs_implanted_rv # Code

seqs_implanted_motifs_rv # Code

seqs_implanted_motifs_top_rv # Code

print("Le motif original est : ", motif)

print("Le motif reverse est : ", motif_rv)

print(seqs_implanted_motifs_top_rv)

9\. Sauvegardez vos séquenecs artificielles originales et implantées, ainsi que le motif généré pour les reutiliser les séances suivantes.

In [ ]:
seqs_file = "seqs_artificielles.txt"

seqs_impl_file = "seqs_artificielles_impl.txt"

motif_file = "motif.txt"

# Code


## Partie B : Recherche de motifs identiques sur le génome de _C. glabrata_

1\. Le fichier **C_glabrata_1000bp_only.fasta** contiens les séquences régulatrices pour tous les gènes de votre organisme. Nous avons pris les 1000bp en amont du codon start. Cherchez les 50 motifs de taille 7 les plus fréquents dans votre génome. Pensez aussi à mesurer le temp utilisé pour obtenir les résultats avec la fonction `time.time()`.

In [ ]:
import time

k = 7
top = 50

def readFasta(fastaFileName):
    """
    Lit un fichier fasta
    entrée fastaFileName: nom du fichier fasta
    sortie séquences: liste contenant toutes les séquences du fichier
    """
    
    sequence = ""
    sequences_list = []
    prev_header = ""
    header = ""
        
    for line in open(fastaFileName):
        string = line.strip()
        if string[0] != ">":
            if prev_header != header:
                prev_header = header
            sequence = sequence + string
        else:
            header = string
            if sequence != "":
                sequences_list.append(sequence)
                sequence = ""

    sequences_list.append(sequence)
    return sequences_list

# Téléchargér le génome
genome = readFasta("C_glabrata_1000bp_only.fasta")

#  Mesurez le temps de calcul
start # Code

genome_motifs # Code

genome_motifs_top # Code

end # Code

print("Top motifs :\n", genome_motifs_top)

print("Temp :", end - start)

2\. Quel sont les trois motifs de taille 7 les plus fréquents? Pensez-vous que ces motifs correspondent à de facteur de transcription connus? Justifiez votre réponse.

<font color='blue'>
Votre réponse
</font>

3\. Le motif **TGATTCAT** correspond au facteur de transcription **Gcn4**, qui est trés fréquemment retrouvé dans le génome des levures. Développez une fonction pour rechercher un motif spécifique dans un dictionnaire de motfis trouvés.

In [ ]:
def searchGivenMotif(motifsTrouve, motifSpecifique, decroissant = True):
    """
    Cherche un motif specifique dans un dictionnaire de motifs trouvés
    entrée motifsTrouve : dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    entrée motifSpecifique: un motif specifique à chercher
    entrée decroissant : bool, si True, le dictionnaire est trié par ordre décroissant de valeur
    sortie fréquence : la fréquence du motif
    sortie ranking : dans quelle position le motif a été trouvé
    >>>searchGivenMotif(test_motifs, "TAT")
    (2, 2)
    """
    
    ranking = 0
    frequence = 0
    
    return ranking, frequence
 
searchGivenMotif(test_motifs, "TAT", decroissant = True)

4\. Essayez votre fonction avec le dictionnaire de motifs de _C. glabrata_. Est-ce que vous avez trouvé le motif de **Gcn4** ? Si oui, à quel ranking et avec quelle fréquence ?

In [ ]:
motifGcn4 = 'TGATTCA'

gcn_rank, gcn_freq # Code

print("Motif Gcn4 trouvé\nRanking : %s\nFréquence : %s" % (gcn_rank, gcn_freq))

5\. Les motifs peu complexes, avec par exemple 5, 6 ou 7 fois la même lettre (homopolymères), sont courants dans les génomes et ils n'ont généralement pas de signification biologique. Faites une fonction pour éliminer les motifs peu complexes, _i. e._ qui ont au moins $m$ fois la même lettre. Faites attention au type d'objet d'entrée pour chaque fonction.

In [ ]:
def removeLowComplexeHomo(motifs:list, m:int):
    """
    Enlève les motifs peu complexe ayant m fois le même nucléotide
    entrée motifs: liste de motifs
    entrée m: taille de repetition de nucléotide
    sortie motifsClean: liste de motifs sans les motifs peu complexe
    >>>removeLowComplexeHomo(['TAA', 'AAG', 'AGT', 'GTA', 'TAT', 'ATA', 'CTA', 'ATC'], 2)
    ['AGT', 'GTA', 'CTA', 'ATC']
    """

    motifsClean = []

    return motifsClean

test_motifs_list # Code

removeLowComplexeHomo(test_motifs_list, 2)

6\. **BONUS :** Dans le context de ce projet, vous pouvez eliminer aussi les motifis ayant deux lettre répétés, comme par exemple **AGAGAGA** (heteropolymères). Vous pouvez aussi penser à supprimer heteropolymères avec une variation, par example **AGAGTAG**. Créez une fonction pour enlever ce type de motifs peu complex dans le dictionnaire test ci-dessou. Réfléchissez aux meilleurs paramètres pour reproduire le résultat exemple dans la description de la fonction.

In [ ]:
test_motifs_complex_list = ['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'AGGGTGG', 'TGGCTTA', 'AGAGTAG', 'GCCCCTC']

import re # TME1.2

def removeLowComplexeHetero(motifs:list, n:int, variation = "yes"):
    """
    Enlève les motifs peu complexe ayant n fois un dinucléotide
    entrée motifs: liste de motifs, clé = motif, valeur = fréquence d'observation
    entrée n: taille de répétition de dinucléotide
    entrée variation : string, si "yes" permettre variation d'un nucléotide
    sortie motifsClean: liste de motifs sans les motifs peu complexe
    >>>removeLowComplexeHetero(['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'AGGGTGG', 'TGGCTTA', 'AGAGTAG', 'GCCCCTC'], 3, variation = "yes")
    ['GGTTTGG', 'TGAGTTA', 'TGCCGTG', 'AGAGAGA', 'TCACCGA', 'TTGGTAT', 'TGGCTTA', 'AGAGTAG']
    """
    
    motifsClean = []

    return motifsClean

Tests et résultats attendus.

In [ ]:
test_motifs_complex_homo # Code

# test_motifs_complex_homo_hetero # Code, si obtenu

# test_motifs_complex_homo_hetero_var

# test_motifs_complex_homo_hetero_var_2

print(test_motifs_complex_homo)
# Affichez, si obtenu
#print(test_motifs_complex_homo_hetero)
#print(test_motifs_complex_homo_hetero_var)
#print(test_motifs_complex_homo_hetero_var_2)

7\. Filtrez les motif peu complexes dans votre génome. Combien de motifs y avait-il dans le dictionnaire original et combien en reste-t-il après chaque filtrage ?

In [ ]:
# Attention au type d'objet

genome_motifs_list # Code
print("K-mers : %s" % (len(genome_motifs)))

genome_motifs_list # Code
print("K-mers sans homopolymères : %s" % (len(genome_motifs_list)))

# BONUS
#genome_motifs_list # Code, si obtenu
#print("K-mers sans heteropolymères (variation = 'no') : %s" % (len(genome_motifs_list)))

# BONUS
#genome_motifs_list # Code, si obtenu
#print("K-mers sans heteropolymères (variation = 'yes') : %s" % (len(genome_motifs_list)))

8\. Une fois que vous avez appris que les motifs peu complexes emmenent à recherches inutiles, créez une fonction amelliorée de `searchMotifs()` qui prend comme entrée une liste de séquences et une liste de motifs filtrée (sans peu complexes). Cette fonction abandone les recherches inutiles et considére juste les motifd dans la liste donée.

In [ ]:
def hashTable(sequences:list, kmersV:list):
    """
    Cherche les motifs de taille k dans un ensemble de séquences avec la méthode de Hash Table
    entrée séquences : liste de séquences
    entrée kmersV: liste de Kmers valides à chercher
    sortie motifs_freq_dict: dictionnaire de motifs, clé = motif, valeur = fréquence d'observation
    >>>hashTable(['TAAGTAA', 'TATAA', 'CTATC'], ['TAA', 'AAG', 'AGT', 'GTA', 'TAT', 'ATA', 'CTA', 'ATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    
    motifs_freq_dict  = {}
    
    return motifs_freq_dict

hashTable(['TAAGTAA', 'TATAA', 'CTATC'], ['TAA', 'AAG', 'AGT', 'GTA', 'TAT', 'ATA', 'CTA', 'ATC'])

9\. Maintenant, cherchez les 50 motifs de taille 7 les plus fréquents dans les deux brin de votre génome. Utilisez la liste de motifs (k-mers) créé precedement. Est-ce que cela donne le même résultat dans les deux brins ? Dans quel position vous trouvez le motif **Gcn4** après utiliser la fonction modifiée `hashTable()` dans chaque brin ? Justifiez votre réponse.

**Brin 1**

In [ ]:
genome_motifs # Code

genome_motifs_top # Code

print("Top motifs :\n", genome_motifs_top)

**Brin 2**

In [ ]:
# Générer les séquences reverse complement
genome_rev # Code

genome_motifs_rev # Code

genome_motifs_rev_top # Code

print("Top motifs :\n", genome_motifs_rev_top)

In [ ]:
# Affiche résultats
print("Brin 1")
gcn_rank_filt, gcn_freq_filt # Code
print("Motif Gcn4 trouvé\nRanking : %d\nFrequence : %d" % (gcn_rank_filt, gcn_freq_filt))

print("\n")

print("Brin 2")
gcn_rank_filt, gcn_freq_filt # Code
print("Motif Gcn4 trouvé\nRanking : %d\nFrequence : %d" % (gcn_rank_filt, gcn_freq_filt))

<font color='blue'>
Votre réponse
</font>

10\. **BONUS**: Développez une fonction qui, à partir d'un dictionnaire de résultats déjà trié, regroupe les résultats du motif reverse complémentaire afin de faciliter la comparaison.

In [ ]:
def gatherRevCompMotifs(motifs_dict):
    """
    Trouver les motifs reverse complementaire et rassembler leur résultats
    entrée motifs_dict : dictionaire de résultats de recherche de motifs
    sortie motifs_dict_gather : dictionaire de motifs, clé = motif, valeur = valeur motif fw, valeur motif rv
    """
    
    motifs_dict_gather = {}
    
    return motifs_dict_gather

genome_motifs_gat = gatherRevCompMotifs(genome_motifs_rev)

print("Dictionnare rassemblé:")

gcn_rank_filt, gcn_freq_filt # Code

print("Motif Gcn4 trouvé\nRanking : %d\nFrequence : %s" % (gcn_rank_filt, gcn_freq_filt))

## Partie C : Recherche de motifs identiques sur vos données

1\. Le fichier "Sequence_by_Peaks_##.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquez les fonctions precedents pour chercher les 3 motifs les plus fréquents dans les deux brim. Il faut bien evidement enlever les motifs peu complexe. Essayez plusier tailles de motifs.

In [ ]:
k = 7 # tester d'autres tailles k = 6, k = 8, etc.

peaks # Code

peaks_motifs_list # Code

peaks_motifs # Code

peaks_motifs_gat # Code

peaks_motifs_top = getTopMotifs(peaks_motifs_gat, top)

print("Top motifs :\n", peaks_motifs_top)

2\. Ulitsez la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ?

<font color='blue'>
Votre réponse
</font>